In [1]:
from ultralytics import YOLO
from yolo_config import Config
import os
import pandas as pd

metrics_columns = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 
                   'metrics/mAP50-95(B)', 'fitness']

In [2]:
# Config.set_local_settings()
Config.set_gpu_settings()

for model_size in Config.MODEL_SIZES:
    model_dir = os.path.join(Config.RUNS_DIR, Config.DATASET, model_size)
    model_metrics = pd.DataFrame(columns=metrics_columns)
    model_metrics.index.name = 'threshold'
    
    threshold_metrics_by_fold = {
        t:pd.DataFrame(columns=metrics_columns) for t in Config.THRESHOLDS
    }
    
    for threshold in Config.THRESHOLDS:
        threshold_results = pd.DataFrame(columns=metrics_columns)
        threshold_results.index.name = 'fold'
        
        for fold in Config.FOLDS:

            # Weights From Training

            fold_name = f'fold_{fold}'
            threshold_name = f'conf={threshold}'
            
            best_weights_path = os.path.join(model_dir, fold_name, 'train', 'weights','best.pt')
            model = YOLO(best_weights_path)

            # Validate for every threshold

            yaml_path = os.path.join(Config.DATASET, fold_name, 'data.yaml')
            validation_project = os.path.join(model_dir, fold_name, threshold_name)
            metrics = model.val(
                data = yaml_path, 
                split = 'val', 
                conf = threshold, 
                project = validation_project,
            )
            
            threshold_results.loc[fold] = pd.Series(metrics.results_dict)

        model_metrics.loc[threshold] = threshold_results.mean()
        
    model_metrics_path = os.path.join(model_dir, Config.METRICS_FILE)
    model_metrics.to_csv(model_metrics_path)
